In [18]:
import pandas as pd
import os
import multiprocessing as mp
import time
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

In [19]:
index_filepath_eng = "F:/Environmental Baseline Data/Version 4 - Final/Indices/esa_index_vcs_ENG_3aug2021.csv"


In [20]:
# =============================== Prepare index dataframe ===============================
df_index_raw = pd.read_csv(index_filepath_eng)

# Create a temporary column in the index dataframe as table identification
df_table_id = df_index_raw.groupby(['Title', 'Data ID']).size()\
    .reset_index().drop(columns=[0])\
    .reset_index().rename(columns={'index': 'ID'})
df_index_raw = df_index_raw.merge(df_table_id, left_on=['Title', 'Data ID'], right_on=['Title', 'Data ID'])


In [21]:
# Remove bad csvs
df_index = df_index_raw[~df_index_raw['bad_csv']]

In [22]:
assert len(df_index[df_index['bad_csv'] == True]) == 0

In [23]:
# Add a new column - Project Download Path
df_index['Project Download Path'] = df_index['Download folder name']\
    .apply(lambda x: '/projects/{}.zip'.format(x))

<ipython-input-23-f03c43fac486>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_index['Project Download Path'] = df_index['Download folder name']\


In [24]:
# Add a new column - Table Download Path
df_table_filename = df_index.sort_values(['PDF Page Number'])\
    .groupby('ID')['filename'].first().reset_index().rename(columns={'filename': 'Table Name'})
df_table_filename['Table Name'] = df_table_filename['Table Name']\
    .apply(lambda x: x.replace('.csv', '').replace('--', '-'))
df_index = df_index.merge(df_table_filename, left_on='ID', right_on='ID')
df_index['Table Download Path'] = df_index['Table Name']\
    .apply(lambda x: '/tables/{}.zip'.format(x))

In [25]:
df_index.head()

,Unnamed: 0,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,bad_csv,filename,old_filename,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples,ID,Project Download Path,Table Name,Table Download Path
0,0,9134,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-ab...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv,0,0,0,17,15,0,1,0,0,0,0,0,0,0,0,14,1,0,0,0,0,26,586,/projects/nrthmntn.zip,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2,/tables/nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.zip
1,1,9135,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE (AITKEN CREEK SECTION),Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv,3,2,6,47,89,3,13,16,1,0,0,0,0,0,25,22,2,0,1,20,2,22,995,/projects/nrthmntn.zip,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2,/tables/nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.zip
2,2,9136,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE (AITKEN CREEK SECTION),Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/1060040,A3Q

In [26]:
# Remove ',All' from Pipeline Location, ESA Section(s) Topics
df_index['Pipeline Location'] = df_index['Pipeline Location']\
    .apply(lambda x: ', '.join([location for location in x.split(', ') if location != 'All']))
df_index['ESA Section(s) Topics'] = df_index['ESA Section(s) Topics']\
    .apply(lambda x: ', '.join([section for section in x.split(', ') if section != 'All']) if type(x) is str else x)

In [27]:
# Update ESA Download URLs
df_index['ESA Folder URL'] = df_index['ESA Folder URL'].apply(lambda x: x.replace('LoadResult', 'View'))

In [28]:
# Prepare a list of column names for the final index files
columns_index = [col for col in df_index.columns.to_list() if col not in (
    'Table Name', 'CSV Download URL', 'Download folder name', 'Zipped Project Link', 'Unnamed: 0',
    'Unnamed: 0.1', 'Index', 'filename', 'old_filename', 'Data ID', 'bad_csv')]

In [29]:
df_index.head()

,Unnamed: 0,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,bad_csv,filename,old_filename,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples,ID,Project Download Path,Table Name,Table Download Path
0,0,9134,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),British Columbia,GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,Water,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-3-summary-of-aquatics-field-work-and-ab...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.csv,1059614_14_lattice-v_1.csv,0,0,0,17,15,0,1,0,0,0,0,0,0,0,0,14,1,0,0,0,0,26,586,/projects/nrthmntn.zip,nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2,/tables/nrthmntn_table-3-summary-of-aquatics-field-work-and-aborigi_pt-1_pg-14_doc-num-A3Q6H2.zip
1,1,9135,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE (AITKEN CREEK SECTION),Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/1059614,Large Projects (over 40 km),British Columbia,GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Consultants",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060220,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3890551,Appendix G: TERA Aquatics Summary Report,15.0,Water,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrthmntn_table-4-summary-of-watercourse-crossings-alon...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.csv,1059614_17_lattice-v_1.csv,3,2,6,47,89,3,13,16,1,0,0,0,0,0,25,22,2,0,1,20,2,22,995,/projects/nrthmntn.zip,nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2,/tables/nrthmntn_table-4-summary-of-watercourse-crossings-along-the_pt-1_pg-17_doc-num-A3Q6H2.zip
2,2,9136,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE (AITKEN CREEK SECTION),Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1060040,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS

In [ ]:

# =============================== Create Master Index File ===============================
# Added figures back to alpha index file
df_index_with_figure = pd.read_excel('//luxor/data/Branch/Environmental Baseline Data/Version 4 - Final/Indices/ESA_website_ENG.xlsx')
figure_columns = columns_index.copy()
figure_columns.remove('ID')
figure_columns.remove('Project Download Path')
figure_columns.remove('Table Download Path')
df_figure = df_index_with_figure[df_index_with_figure['Content Type'] == 'Figure'][figure_columns]
df_figure['ID'] = df_figure.index + df_index_raw['ID'].max() + 1
df_figure['Application Filing Date'] = df_figure['Application Filing Date'].apply(lambda x: x.strftime('%Y-%m-%d'))

# Add bad tables
bad_table_columns = columns_index.copy()
bad_table_columns.remove('Project Download Path')
bad_table_columns.remove('Table Download Path')
df_table_bad = df_index_raw[df_index_raw['bad_csv']].sort_values(['ID', 'PDF Page Number']).groupby('ID').first()\
    .reset_index()[bad_table_columns]
df_table_bad['Good Quality'] = False

# Concatenate all tables
df_table = df_index.sort_values(['ID', 'PDF Page Number']).groupby('ID').first()\
    .reset_index()[columns_index]
df_table['Good Quality'] = True

df_index_new = pd.concat([df_figure, df_table, df_table_bad])

# Remove ',All' from Pipeline Location, ESA Section(s) Topics
df_index_new['Pipeline Location'] = df_index_new['Pipeline Location']\
    .apply(lambda x: ', '.join([location for location in x.split(', ') if location != 'All']))
df_index_new['ESA Section(s) Topics'] = df_index_new['ESA Section(s) Topics']\
    .apply(lambda x: ', '.join([section for section in x.split(', ') if section != 'All']) if type(x) is str else x)
# Update ESA Download URls
df_index_new['ESA Folder URL'] = df_index_new['ESA Folder URL'].apply(lambda x: x.replace('LoadResult', 'View'))

# Export alpha index
df_index_new.to_csv(os.path.join(new_folder, 'ESA_website_ENG.csv'), index=False)


# Quality check

df_index_new = pd.read_csv(os.path.join(new_folder, 'ESA_website_ENG.csv'))

for project_path in df_index_new['Project Download Path'].unique():
    if type(project_path) is str:
        project_folder = project_path.split('/')[-1]
        if project_folder not in os.listdir(new_folder_projects):
            print('Missing project folder: {}'.format(project_folder))

for table_path in df_index_new['Table Download Path']:
    if type(table_path) is str:
        table_folder = table_path.split('/')[-1]
        if table_folder not in os.listdir(new_folder_tables):
            print('Missing table folder: {}'.format(table_folder))

df_index_new['table missing'] = df_index_new['Table Download Path']\
    .apply(lambda x: x.split('/')[-1] not in os.listdir(new_folder_tables) if type(x) is str else x)

for table_id in df_index_new[df_index_new['table missing'] == True]['ID']:
    print(table_id)
    bundle_for_table(df_index, table_id, new_folder_tables, csv_file_folder, columns_index, readme_table_filepath)

for table_path in df_index_new[df_index_new['table missing'] == True]['Table Download Path']:
    if type(table_path) is str:
        table_folder = table_path.split('/')[-1]
        if table_folder not in os.listdir(new_folder_tables):
            print('Missing table folder: {}'.format(table_folder))

# Check if certain attributes stay the same for projects
df = pd.read_csv('G:/ESA_downloads/download_Bingjie_Mar262021/ESA_website_ENG.csv')
df_fra = pd.read_csv('G:/ESA_downloads/download_Bingjie_May062021_fra/ESA_website_FRA.csv', encoding='latin-1')

df_project = df.groupby(['Application Short Name'])[[
    'Application Name',
    'Application Filing Date',
    'Company Name',
    'Commodity',
    'Application Type (NEB Act)',
    'Pipeline Location',
    'Hearing order',
    'Consultant Name',
    'Pipeline Status',
    'Application URL',
    'Project Download Path'
    ]].nunique()

df_project_fra = df_fra.groupby(['Nom abrégé de la demande'])[[
    'Nom de la demande',
    'Dépôt de la demande',
    'Nom de la société',
    'Produit de base',
    'Type de demande (Loi sur l\'Office national de l\'énergie)',
    'Emplacement du pipeline',
    'Ordonnance d\'audience',
    'Nom du consultant',
    'État d\'avancement',
    'URL de la demande',
    "Chemin d'accès pour télécharger le projet"
]].nunique()
